<div dir="ltr" style="text-align: left;" trbidi="on">
<h2 style="text-align: left;">
100 Days of ML Day 67</h2>
<h3 style="text-align: left;">
HackerEarth Airplane Accident Severity Challenge Part 4</h3>
Built a Deep Neural Network to classify severity of the Airplane Accident. Accuracy is around 94% on the dev set and got a 0.84 score on the competition test set which is not an improvement on the Gradient Boosting Model.<br />
<br />
Code: <a href="https://github.com/hithesh111/Hith100/blob/master/100Days/day067.ipynb">https://github.com/hithesh111/Hith100/blob/master/100Days/day067.ipynb</a><br />
<br />
Competition: <a href="https://www.hackerearth.com/challenges/competitive/airplane-accident-severity-hackerearth-machine-learning-challenge/">https://www.hackerearth.com/challenges/competitive/airplane-accident-severity-hackerearth-machine-learning-challenge/</a><br />
<br />
Leaderboard: <a href="https://www.hackerearth.com/challenges/competitive/airplane-accident-severity-hackerearth-machine-learning-challenge/leaderboard/how-severe-can-an-airplane-accident-be-03e7a3f1/">https://www.hackerearth.com/challenges/competitive/airplane-accident-severity-hackerearth-machine-learning-challenge/leaderboard/how-severe-can-an-airplane-accident-be-03e7a3f1/</a></div>


In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from keras import regularizers

Using TensorFlow backend.


In [2]:
df = pd.read_csv("/home/hithesh/Downloads/Airplane Data/train.csv")
df.head()

,Severity,Safety_Score,Days_Since_Inspection,Total_Safety_Complaints,Control_Metric,Turbulence_In_gforces,Cabin_Temperature,Accident_Type_Code,Max_Elevation,Violations,Adverse_Weather_Metric,Accident_ID
0,Minor_Damage_And_Injuries,49.223744,14,22,71.285324,0.272118,78.04,2,31335.476824,3,0.424352,7570
1,Minor_Damage_And_Injuries,62.465753,10,27,72.288058,0.423939,84.54,2,26024.711057,2,0.352350,12128
2,Significant_Damage_And_Fatalities,63.059361,13,16,66.362808,0.322604,78.86,7,39269.053927,3,0.003364,2181
3,Significant_Damage_And_Serious_Injuries,48.082192,11,9,74.703737,0.337029,81.79,3,42771.499200,1,0.211728,5946
4,Significant_Damage_And_Fatalities,26.484018,13,25,47.948952,0.541140,77.16,3,35509.228515,2,0.176883,9054


In [3]:
def normalize(df,colname):
    df[colname]=(df[colname]-df[colname].mean())/df[colname].std()
    
def standardize(df,colname):
    df[colname]=(df[colname]-df[colname].min())/(df[colname].max()-df[colname].min())

def outliercap(df,colname,factor):
    upper_lim = df[colname].mean () + df[colname].std () * factor
    lower_lim = df[colname].mean () - df[colname].std () * factor
    df.loc[(df[colname] > upper_lim),colname] = upper_lim
    df.loc[(df[colname] < lower_lim),colname] = lower_lim

In [4]:
for i in [1,4,5,6,8,10]:
    normalize(df,df.columns[i])
for i in [2,3,7,9]:
    standardize(df,df.columns[i])
df['Severity'] = df['Severity'].apply(lambda x: 0 if x=='Minor_Damage_And_Injuries' else (1 if x=='Significant_Damage_And_Fatalities' else(2 if x=='Significant_Damage_And_Serious_Injuries' else (3 if x=='Highly_Fatal_And_Damaging' else -1)) ))
print(df.head())

   Severity  Safety_Score  Days_Since_Inspection  Total_Safety_Complaints  \
0         0      0.455280               0.590909                 0.407407   
1         0      1.275824               0.409091                 0.500000   
2         1      1.312608               0.545455                 0.296296   
3         2      0.384543               0.454545                 0.166667   
4         1     -0.953794               0.545455                 0.462963   

   Control_Metric  Turbulence_In_gforces  Cabin_Temperature  \
0        0.516707              -0.901704          -0.699099   
1        0.601092               0.349905           1.656196   
2        0.102457              -0.485492          -0.401969   
3        0.804382              -0.366574           0.659725   
4       -1.447149               1.316111          -1.017970   

   Accident_Type_Code  Max_Elevation  Violations  Adverse_Weather_Metric  \
0            0.166667      -0.070645         0.6                0.442679   
1     

In [5]:
factor = 3
outliercap(df,'Safety_Score',factor)
outliercap(df,'Max_Elevation',factor)
outliercap(df,'Control_Metric',factor)
outliercap(df,'Turbulence_In_gforces',factor)

In [6]:
labels = df['Severity'].unique()
y = df['Severity']
features = df.columns[1:-1]
print(features)
X = df[features]
print(X.head())

Index(['Safety_Score', 'Days_Since_Inspection', 'Total_Safety_Complaints',
       'Control_Metric', 'Turbulence_In_gforces', 'Cabin_Temperature',
       'Accident_Type_Code', 'Max_Elevation', 'Violations',
       'Adverse_Weather_Metric'],
      dtype='object')
   Safety_Score  Days_Since_Inspection  Total_Safety_Complaints  \
0      0.455280               0.590909                 0.407407   
1      1.275824               0.409091                 0.500000   
2      1.312608               0.545455                 0.296296   
3      0.384543               0.454545                 0.166667   
4     -0.953794               0.545455                 0.462963   

   Control_Metric  Turbulence_In_gforces  Cabin_Temperature  \
0        0.516707              -0.901704          -0.699099   
1        0.601092               0.349905           1.656196   
2        0.102457              -0.485492          -0.401969   
3        0.804382              -0.366574           0.659725   
4       -1.447149   

In [7]:
l1 = tf.keras.layers.Dense(units=64,input_shape=[len(features)],activation = tf.nn.relu,kernel_regularizer=regularizers.l2(5e-4))
l2 = tf.keras.layers.Dense(units=256,activation = tf.nn.relu,kernel_regularizer=regularizers.l2(5e-4))
l3 = tf.keras.layers.Dense(units=512,activation = tf.nn.relu,kernel_regularizer=regularizers.l2(5e-4))
l4 = tf.keras.layers.Dense(units=512,activation = tf.nn.relu,kernel_regularizer=regularizers.l2(5e-4))
l5 = tf.keras.layers.Dense(units=256,activation = tf.nn.relu,kernel_regularizer=regularizers.l2(5e-4))
l6 = tf.keras.layers.Dense(units=4,activation = tf.nn.softmax)

In [8]:
model = tf.keras.Sequential([l1,l2,l3,l4,l5,l6])

In [9]:
model.compile(optimizer = 'adam',loss='sparse_categorical_crossentropy',metrics = ['accuracy'])

In [10]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.20, random_state=36)

In [11]:
model.fit(X_train, y_train, epochs=70,verbose=True)

Train on 8000 samples
Epoch 1/70
8000/8000 [==============================] - 2s 216us/sample - loss: 1.2550 - accuracy: 0.5960
Epoch 2/70
8000/8000 [==============================] - 1s 175us/sample - loss: 0.6710 - accuracy: 0.8311
Epoch 3/70
8000/8000 [==============================] - 1s 187us/sample - loss: 0.4921 - accuracy: 0.8917
Epoch 4/70
8000/8000 [==============================] - 1s 173us/sample - loss: 0.4232 - accuracy: 0.9056
Epoch 5/70
8000/8000 [==============================] - 1s 170us/sample - loss: 0.3861 - accuracy: 0.9154
Epoch 6/70
8000/8000 [==============================] - 1s 169us/sample - loss: 0.3468 - accuracy: 0.9256
Epoch 7/70
8000/8000 [==============================] - 1s 170us/sample - loss: 0.3245 - accuracy: 0.9319
Epoch 8/70
8000/8000 [==============================] - 1s 186us/sample - loss: 0.3215 - accuracy: 0.9296
Epoch 9/70
8000/8000 [==============================] - 1s 177us/sample - loss: 0.3148 - accuracy: 0.9311
Epoch 10/70
8000/8000 [=

In [12]:
model.evaluate(X_valid,y_valid)

2000/2000 [==============================] - 0s 107us/sample - loss: 0.2488 - accuracy: 0.9380


[0.248780759871006, 0.938]

In [13]:
model.fit(X, y, epochs=50,verbose=True)

Train on 10000 samples
Epoch 1/50
10000/10000 [==============================] - 2s 195us/sample - loss: 0.2038 - accuracy: 0.9489
Epoch 2/50
10000/10000 [==============================] - 2s 170us/sample - loss: 0.1997 - accuracy: 0.9510
Epoch 3/50
10000/10000 [==============================] - 2s 172us/sample - loss: 0.1951 - accuracy: 0.9532
Epoch 4/50
10000/10000 [==============================] - 2s 172us/sample - loss: 0.1912 - accuracy: 0.9508
Epoch 5/50
10000/10000 [==============================] - 2s 170us/sample - loss: 0.1947 - accuracy: 0.9529
Epoch 6/50
10000/10000 [==============================] - 2s 171us/sample - loss: 0.1942 - accuracy: 0.9511
Epoch 7/50
10000/10000 [==============================] - 2s 172us/sample - loss: 0.2016 - accuracy: 0.9482
Epoch 8/50
10000/10000 [==============================] - 2s 188us/sample - loss: 0.1885 - accuracy: 0.9550
Epoch 9/50
10000/10000 [==============================] - 2s 173us/sample - loss: 0.1881 - accuracy: 0.9562
Epoch

In [14]:
def inv_transform(M):
    N=[]
    for i in range(M.shape[0]):
        if(max(M[i])==M[i][0]):
            N.append('Minor_Damage_And_Injuries')
        if(max(M[i])==M[i][1]):
            N.append('Significant_Damage_And_Fatalities')
        if(max(M[i])==M[i][2]):
            N.append('Significant_Damage_And_Serious_Injuries')
        if(max(M[i])==M[i][3]):
            N.append('Highly_Fatal_And_Damaging')
    return N

In [15]:
df2 = pd.read_csv('/home/hithesh/Downloads/Airplane Data/test.csv')
acc_id = df2['Accident_ID']
for i in [0,3,4,5,7,9]:
    normalize(df2,df2.columns[i])
for i in [1,2,6,8]:
    standardize(df2,df2.columns[i])
X = df2[features]
pred = model.predict(X)
df2['Severity'] = pd.Series(inv_transform(pred))
final = pd.concat([acc_id,df2['Severity']],axis=1)
print(final)
print(df2['Severity'].unique())
final.to_csv('PlaneSubmission.csv',header=True , index=False)

      Accident_ID                                 Severity
0               1                Highly_Fatal_And_Damaging
1              10        Significant_Damage_And_Fatalities
2              14  Significant_Damage_And_Serious_Injuries
3              17                Highly_Fatal_And_Damaging
4              21        Significant_Damage_And_Fatalities
5              40        Significant_Damage_And_Fatalities
6              45        Significant_Damage_And_Fatalities
7              50                Highly_Fatal_And_Damaging
8              52                Highly_Fatal_And_Damaging
9              85  Significant_Damage_And_Serious_Injuries
10             95  Significant_Damage_And_Serious_Injuries
11            106  Significant_Damage_And_Serious_Injuries
12            111                Highly_Fatal_And_Damaging
13            123        Significant_Damage_And_Fatalities
14            129        Significant_Damage_And_Fatalities
15            130  Significant_Damage_And_Serious_Injuri